In [1]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [11]:
csv_train = tf.data.experimental.make_csv_dataset(
    './../Naive_Bayes/tweets/allLabeledTweets.csv',
    batch_size=400,
    select_columns=['message_lowercase', 'label'],    
    label_name='label',
    num_epochs=1,
    ignore_errors=True,)

csv_val = tf.data.experimental.make_csv_dataset(
    './../Naive_Bayes/tweets/allLabeledTweets_copy.csv',
    batch_size=400,
    select_columns=['message_lowercase', 'label'],    
    label_name='label',
    num_epochs=1,
    ignore_errors=True,)

csv_test = tf.data.experimental.make_csv_dataset(
    './../Naive_Bayes/tweets/allLabeledTweets_copy_copy.csv',
    batch_size=400,
    select_columns=['message_lowercase', 'label'],    
    label_name='label',
    num_epochs=1,
    ignore_errors=True,)


In [ ]:
# train_neutral = train_df[train_df["label"] == 0]
# train_positive = train_df[train_df["label"] == 1]
# train_negative = train_df[train_df["label"] == 2]

# train_df = pd.concat([
#     train_neutral.sample(87),
#     train_positive,
#     train_negative.sample(87)])

In [3]:
preprocess_url = 'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3'
encoder_url = 'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4'

bert_preprocess_model = hub.KerasLayer(preprocess_url)

In [4]:
text_test = ['Sveiki! Informējam, ka saistībā ar tehniskiem uzlabojumiem šonakt laika posmā no plkst. 2.00 līdz 4.00 BITES tīklā īslaicīgi – aptuveni 3 min – nebūs pieejams internets. Ja tas nav pieejams ilgāku laiku, aicinām restartēt ierīces. Atvainojamies par sagādātajām neērtībām!']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_word_ids', 'input_type_ids', 'input_mask']
Shape      : (1, 128)
Word Ids   : [  101 77439 20897   106 43964 19341 19542 10147   117 10730 13410 10562]
Input Mask : [1 1 1 1 1 1 1 1 1 1 1 1]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [5]:
bert_model = hub.KerasLayer(encoder_url)

In [6]:
bert_results = bert_model(text_preprocessed)

print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Pooled Outputs Shape:(1, 768)
Pooled Outputs Values:[ 0.23613809 -0.04442507  0.23505516 -0.15770182 -0.03203802  0.2451003
  0.09333649  0.13524106 -0.20804942  0.14433675  0.08714208 -0.15120143]
Sequence Outputs Shape:(1, 128, 768)
Sequence Outputs Values:[[-0.27232003  0.03256002 -0.04421538 ...  0.09869212  0.35841888
  -0.14795616]
 [-0.66119045  0.03879939  0.28476703 ... -0.34495023  0.92087734
  -0.09457634]
 [-0.3153756  -0.16313021  0.3829687  ... -0.22015034  0.7926136
  -0.6949453 ]
 ...
 [-0.68508047  0.3063221   0.47708553 ... -0.11326828  0.6556493
  -0.10139327]
 [-0.2931501  -0.71528256 -0.37650025 ... -1.026762    1.0475296
  -0.09604168]
 [-0.02384613 -1.0335578  -0.11690557 ...  0.729188    0.92268795
  -0.96160156]]


In [7]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='message_lowercase')
    preprocessing_layer = hub.KerasLayer(preprocess_url, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(encoder_url, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
    return tf.keras.Model(text_input, net)

In [8]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
print(tf.sigmoid(bert_raw_result))

tf.Tensor([[0.3996465]], shape=(1, 1), dtype=float32)


In [14]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
# metrics = tf.metrics.BinaryAccuracy()
metrics = [
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

In [15]:
epochs = 5
steps_per_epoch = 3
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [16]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [17]:
print(f'Training model with {encoder_url}')
history = classifier_model.fit(x=csv_train,
                               validation_data=csv_val,
                               epochs=epochs)

Training model with https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4
Epoch 1/5


InvalidArgumentError:  assertion failed: [predictions must be >= 0] [Condition x >= y did not hold element-wise:] [x (model/classifier/BiasAdd:0) = ] [[0.636165][-0.331228614][0.530802488]...] [y (Cast_3/x:0) = ] [0]
	 [[node assert_greater_equal/Assert/AssertGuard/Assert
 (defined at C:\Users\aligo\Documents\DEV\twitter-lv-market\venv38\lib\site-packages\keras\utils\metrics_utils.py:608)
]] [Op:__inference_train_function_139908]

Errors may have originated from an input operation.
Input Source operations connected to node assert_greater_equal/Assert/AssertGuard/Assert:
In[0] assert_greater_equal/Assert/AssertGuard/Assert/assert_greater_equal/All:	
In[1] assert_greater_equal/Assert/AssertGuard/Assert/data_0:	
In[2] assert_greater_equal/Assert/AssertGuard/Assert/data_1:	
In[3] assert_greater_equal/Assert/AssertGuard/Assert/data_2:	
In[4] assert_greater_equal/Assert/AssertGuard/Assert/model/classifier/BiasAdd:	
In[5] assert_greater_equal/Assert/AssertGuard/Assert/data_4:	
In[6] assert_gr

In [ ]:
loss, accuracy = classifier_model.evaluate(csv_test)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

In [ ]:
history_dict = history.history
print(history_dict.keys())

acc = history_dict['binary_accuracy']
val_acc = history_dict['val_binary_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

plt.subplot(2, 1, 1)
# r is for "solid red line"
plt.plot(epochs, loss, 'r', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
# plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

In [ ]:
saved_model_path = './twitter_bert_lowercase'

classifier_model.save(saved_model_path, include_optimizer=False)

In [ ]:
reloaded_model = tf.saved_model.load(saved_model_path)

In [ ]:
def print_my_examples(inputs, results):
    result_for_printing = \
    [f'input: {inputs[i]:<30} : score: {results[i][0]:.6f}'
                         for i in range(len(inputs))]
    print(*result_for_printing, sep='\n')
    print()


examples = [
    'Tā ir brīnišķīga filma',
    'Filma bija laba.',
    'Filma bija meh.',
    'Filma bija OK',
    'Tā filma bija briesmīga...'
]

reloaded_results = tf.sigmoid(reloaded_model(tf.constant(examples)))
original_results = tf.sigmoid(classifier_model(tf.constant(examples)))

print('Results from the saved model:')
print_my_examples(examples, reloaded_results)
print('Results from the model in memory:')
print_my_examples(examples, original_results)